## Flight crew scheduling demo using CVXPY, DWave BQM solver, CQM solver and QAOA using Qiskit

Budget Airways is required to assign its crews based in New York to cover all the upcoming scheduled flights. There are many possible sequences of flights that are feasible for a crew to choose from, assuming one crew can only be assigned to one sequence. There are 10 flights and 8 feasible sequences of flights and associated costs given. The problem is to find crew assignment that covers all 10 flights at a minimal total cost.

In [2]:
# pip install cvxopt

In [1]:
import numpy as np

In [3]:
import pandas as pd
import cvxpy as cp

df=pd.read_csv("flight_schedule.csv", index_col = 0, header=1)
df


,1,2,3,4,5,6,7,8
Requirement (Flight),,,,,,,,
New York to Buffalo,1,0,0,1,0,0,1,0
New York to Cincinnati,0,1,0,0,1,0,0,0
New York to Chicago,0,0,1,0,0,1,0,1
Buffalo to Chicago,2,0,0,2,0,0,0,0
Chicago to Cincinnati,0,0,2,3,0,2,0,0
Cincinnati to Pittsburgh,0,2,0,4,0,3,0,0
Cincinnati to Buffalo,0,0,3,0,2,0,0,0
Buffalo to New York,0,0,4,0,3,0,2,0
Pittsburgh to New York,0,3,0,5,0,4,0,0


In [4]:
schedule = df.iloc[:-2]
cost = df.iloc[-2]
hours = df.iloc[-1]

To formulate this problem, we begin by forming a node-arc incidence matrix A, having 
an entry of "1 " if a sequence of flight covers a certain flight and "0" otherwise.

In [5]:
schedule = schedule.applymap(lambda x: 1 if x >= 1 else x)
schedule

,1,2,3,4,5,6,7,8
Requirement (Flight),,,,,,,,
New York to Buffalo,1,0,0,1,0,0,1,0
New York to Cincinnati,0,1,0,0,1,0,0,0
New York to Chicago,0,0,1,0,0,1,0,1
Buffalo to Chicago,1,0,0,1,0,0,0,0
Chicago to Cincinnati,0,0,1,1,0,1,0,0
Cincinnati to Pittsburgh,0,1,0,1,0,1,0,0
Cincinnati to Buffalo,0,0,1,0,1,0,0,0
Buffalo to New York,0,0,1,0,1,0,1,0
Pittsburgh to New York,0,1,0,1,0,1,0,0


Input parameters: 
* $a$: 0-1 matrix
* $c_j$: cost for each feasible sequence of flight ($j=1, 2, ..,8$)
* $b_i$: requirement vector ($i=1, 2, ..., 10$)

In [6]:
a = schedule.values
c = cost.values
b = np.ones(len(a))

Decision variable: 

$y_j=1$ or $0$: Whether the flight sequence $j$ is selected or not

In [7]:
y = cp.Variable(len(c), boolean=True)

Objective: minimize the total cost of assigning crews to the selected sequence of flights

$$\min \;\; z=5y_1 + 4y_2 + 4y_3 + 9y_4 + 7y_5 + 8y_6 + 3y_7 + 3y_8$$

In [8]:
obj = cp.Minimize(c @ y)

Constraint: ensure that at least one crew is assigned to each flight.

For example, to make sure that at least one crew is assigned to the first flight, we have constraint:
$$y_1 + y_4 + y_7 \geq 1$$

Generalize this constraint for all 10 flights, we have:

$$
\begin{pmatrix}
  1  &  0  &  0  &  1  &  0  &  0  &  1  &  0 \\
  0  &  1  &  0  &  0  &  1  &  0  &  0  &  0 \\
  0  &  0  &  1  &  0  &  0  &  1  &  0  &  1 \\
  1  &  0  &  0  &  1  &  0  &  0  &  0  &  0 \\
  0  &  0  &  1  &  1  &  0  &  1  &  0  &  0 \\
  0  &  1  &  0  &  1  &  0  &  1  &  0  &  0 \\
  0  &  0  &  1  &  0  &  1  &  0  &  0  &  0 \\
  0  &  0  &  1  &  0  &  1  &  0  &  1  &  0 \\
  0  &  1  &  0  &  1  &  0  &  1  &  0  &  0 \\
  1  &  0  &  0  &  0  &  0  &  0  &  0  &  1 
\end{pmatrix}
\begin{pmatrix}
y_1\\
y_2\\
y_3\\
y_4\\
y_5\\
y_6\\
y_7\\
y_8\\
\end{pmatrix} \geq
\begin{pmatrix}
1\\
1\\
1\\
1\\
1\\
1\\
1\\
1\\
\end{pmatrix}
$$

$y_j=0$ or $1$ $j=1,2,...,8$

In [9]:
constraints = [a @ y >= b]

In [10]:
import cvxopt
prob = cp.Problem(obj, constraints)
prob.solve()

13.0

In [11]:
print (y.value)

[1. 1. 1. 0. 0. 0. 0. 0.]


These values represent that all the flights can be covered with minimum cost of 13 if flights from column 1,2 and 3 of matrix A are selected. 

## D_wave solution

In [17]:
import dimod
from dimod import BinaryQuadraticModel

Initialize a BQM

In [18]:
model = dimod.BinaryQuadraticModel('BINARY')

In [19]:
f=[f'f{i}' for i in range(len(c))]

Add variables and biases (costs) to the BQM

In [20]:
for i in range(len(c)):
    model.add_variable(f[i], c[i])

In [21]:
print(model)

BinaryQuadraticModel({'f0': 5.0, 'f1': 4.0, 'f2': 4.0, 'f3': 9.0, 'f4': 7.0, 'f5': 8.0, 'f6': 3.0, 'f7': 3.0}, {}, 0.0, 'BINARY')


Adding constraints to the model

In [22]:
for i in range(len(a)):
    c1 = [(f[j] , a[i][j]) for j in range(len(c))]
    model.add_linear_inequality_constraint (c1, lb = 1, ub=len(c), lagrange_multiplier = 11, label = 'c1_' + str(i) )

Sampling using classical solver

In [23]:
from dimod.reference.samplers import ExactSolver
sampler = ExactSolver()

In [24]:
result = sampler.sample(model)

result.first

Sample(sample={'f0': 1, 'f1': 1, 'f2': 1, 'f3': 0, 'f4': 0, 'f5': 0, 'f6': 0, 'f7': 0, 'slack_c1_0_0': 1, 'slack_c1_0_1': 1, 'slack_c1_1_0': 1, 'slack_c1_2_0': 1, 'slack_c1_2_1': 1, 'slack_c1_3_0': 1, 'slack_c1_4_0': 1, 'slack_c1_4_1': 1, 'slack_c1_5_0': 1, 'slack_c1_5_1': 1, 'slack_c1_6_0': 1, 'slack_c1_7_0': 1, 'slack_c1_7_1': 1, 'slack_c1_8_0': 1, 'slack_c1_8_1': 1, 'slack_c1_9_0': 1}, energy=13.0, num_occurrences=1)

This result is same as provided by CVXPY. Now, let's solve this using DWave Quantum Annealer. API token was already configured.

In [240]:
from dwave.system import EmbeddingComposite,DWaveSampler
qpu_sampler = EmbeddingComposite(DWaveSampler())

In [241]:
sampleset = qpu_sampler.sample(model, num_reads=1000)

In [243]:
sampleset.first

Sample(sample={'f0': 1, 'f1': 1, 'f2': 1, 'f3': 0, 'f4': 0, 'f5': 0, 'f6': 0, 'f7': 0, 'slack_c1_0_0': 1, 'slack_c1_0_1': 1, 'slack_c1_1_0': 1, 'slack_c1_2_0': 1, 'slack_c1_2_1': 1, 'slack_c1_3_0': 1, 'slack_c1_4_0': 1, 'slack_c1_4_1': 1, 'slack_c1_5_0': 1, 'slack_c1_5_1': 1, 'slack_c1_6_0': 1, 'slack_c1_7_0': 1, 'slack_c1_7_1': 1, 'slack_c1_8_0': 1, 'slack_c1_8_1': 1, 'slack_c1_9_0': 1}, energy=13.0, num_occurrences=1, chain_break_fraction=0.08333333333333333)

This results is also same as provided by CVXPY.

Solving the problem using CQM and classical solver

In [25]:
from dimod import Binary
x = [Binary(f'x_{j}') for j in range(len(c))]

In [26]:
from dimod import ConstrainedQuadraticModel
cqm = ConstrainedQuadraticModel()

In [27]:
cqm.set_objective(sum(c*x))

In [28]:
con = a@x
con[0].linear

{'x_0': 1.0, 'x_1': 0.0, 'x_2': 0.0, 'x_3': 1.0, 'x_4': 0.0, 'x_5': 0.0, 'x_6': 1.0, 'x_7': 0.0}

In [29]:
con_label1 = []
for i in range(len(con)):
    con_temp = cqm.add_constraint(con[i], '>=', 1, label='c1_'+str(i))
    con_label1.append(con_temp)
con_label1

['c1_0',
 'c1_1',
 'c1_2',
 'c1_3',
 'c1_4',
 'c1_5',
 'c1_6',
 'c1_7',
 'c1_8',
 'c1_9']

In [30]:
from dimod import ExactCQMSolver
sampleset = ExactCQMSolver().sample_cqm(cqm)

In [32]:
def postprocess_cqm_results(cqm, sampleset, return_indices, return_energy):
    feasible = sampleset.filter(lambda s: s.is_feasible)
    if feasible:
        best = feasible.first
    else:
        assert len(cqm.constraints) == 1
        # Sort on violation, then energy
        best = sorted(sampleset.data(), key=lambda x: (list(cqm.violations(x.sample).values())[0],
                                                     x.energy))[0]

    assert list(best.sample.keys()) == sorted(best.sample.keys())
    is_selected = np.array([bool(val) for val in best.sample.values()])

    if return_indices:
        routes = np.array([i for i, val in enumerate(is_selected) if val])
    else:
        routes = is_selected

    if return_energy:
        return routes, best.energy
    else:
        return routes    

In [33]:
route, energy = postprocess_cqm_results(cqm, sampleset, return_indices='False', return_energy='False')

In [34]:
route

array([0, 1, 2])

In [35]:
energy

13.0

Solving using QAOA

In [36]:
from qiskit_optimization import QuadraticProgram

In [37]:
qaoa_model = QuadraticProgram("Flight scheduling")

In [38]:
qaoa_model.binary_var_list(len(c), name='q')

[<Variable: q0 (binary)>,
 <Variable: q1 (binary)>,
 <Variable: q2 (binary)>,
 <Variable: q3 (binary)>,
 <Variable: q4 (binary)>,
 <Variable: q5 (binary)>,
 <Variable: q6 (binary)>,
 <Variable: q7 (binary)>]

In [39]:
qaoa_model.minimize(linear=c)

In [40]:
print(qaoa_model.prettyprint())

Problem name: Flight scheduling

Minimize
  5*q0 + 4*q1 + 4*q2 + 9*q3 + 7*q4 + 8*q5 + 3*q6 + 3*q7

Subject to
  No constraints

  Binary variables (8)
    q0 q1 q2 q3 q4 q5 q6 q7



In [41]:
for i in range(len(a)):
    qaoa_model.linear_constraint (linear=a[i], sense='>=', rhs=1, name = 'c21_' + str(i) )

In [42]:
print(qaoa_model.prettyprint())

Problem name: Flight scheduling

Minimize
  5*q0 + 4*q1 + 4*q2 + 9*q3 + 7*q4 + 8*q5 + 3*q6 + 3*q7

Subject to
  Linear constraints (10)
    q0 + q3 + q6 >= 1  'c21_0'
    q1 + q4 >= 1  'c21_1'
    q2 + q5 + q7 >= 1  'c21_2'
    q0 + q3 >= 1  'c21_3'
    q2 + q3 + q5 >= 1  'c21_4'
    q1 + q3 + q5 >= 1  'c21_5'
    q2 + q4 >= 1  'c21_6'
    q2 + q4 + q6 >= 1  'c21_7'
    q1 + q3 + q5 >= 1  'c21_8'
    q0 + q7 >= 1  'c21_9'

  Binary variables (8)
    q0 q1 q2 q3 q4 q5 q6 q7



In [43]:
from qiskit_optimization.converters import QuadraticProgramToQubo
conv = QuadraticProgramToQubo()
problem2 = conv.convert(qaoa_model)

In [47]:
print(problem2.prettyprint())

Problem name: Flight scheduling

Minimize
  508*c21_0@int_slack@0^2 + 1016*c21_0@int_slack@0*c21_0@int_slack@1
  + 508*c21_0@int_slack@1^2 + 508*c21_2@int_slack@0^2
  + 1016*c21_2@int_slack@0*c21_2@int_slack@1 + 508*c21_2@int_slack@1^2
  + 508*c21_4@int_slack@0^2 + 1016*c21_4@int_slack@0*c21_4@int_slack@1
  + 508*c21_4@int_slack@1^2 + 508*c21_5@int_slack@0^2
  + 1016*c21_5@int_slack@0*c21_5@int_slack@1 + 508*c21_5@int_slack@1^2
  + 508*c21_7@int_slack@0^2 + 1016*c21_7@int_slack@0*c21_7@int_slack@1
  + 508*c21_7@int_slack@1^2 + 508*c21_8@int_slack@0^2
  + 1016*c21_8@int_slack@0*c21_8@int_slack@1 + 508*c21_8@int_slack@1^2
  - 1016*q0*c21_0@int_slack@0 - 1016*q0*c21_0@int_slack@1 + 508*q0^2
  + 1060*q0*q3 + 1016*q0*q6 + 44*q0*q7 - 1016*q1*c21_5@int_slack@0
  - 1016*q1*c21_5@int_slack@1 - 1016*q1*c21_8@int_slack@0
  - 1016*q1*c21_8@int_slack@1 + 1016*q1^2 + 2032*q1*q3 + 44*q1*q4 + 2032*q1*q5
  - 1016*q2*c21_2@int_slack@0 - 1016*q2*c21_2@int_slack@1
  - 1016*q2*c21_4@int_slack@0 - 1016*q2*c

In [46]:
op, offset = problem2.to_ising()
print(f"num qubits: {op.num_qubits}, offset: {offset}\n")
print("operator:")
print(op)

num qubits: 20, offset: 4637.5

operator:
273.5 * IIIIIIIIIIIIIIIIIIIZ
+ 517.0 * IIIIIIIIIIIIIIIIIIZI
+ 771.0 * IIIIIIIIIIIIIIIIIZII
+ 1022.5 * IIIIIIIIIIIIIIIIZIII
+ 272.5 * IIIIIIIIIIIIIIIZIIII
+ 1012.0 * IIIIIIIIIIIIIIZIIIII
+ 506.5 * IIIIIIIIIIIIIZIIIIII
+ 263.5 * IIIIIIIIIIIIZIIIIIII
- 254.0 * IIIIIIIIIIIZIIIIIIII
- 254.0 * IIIIIIIIIIZIIIIIIIII
- 254.0 * IIIIIIIIIZIIIIIIIIII
- 254.0 * IIIIIIIIZIIIIIIIIIII
- 254.0 * IIIIIIIZIIIIIIIIIIII
- 254.0 * IIIIIIZIIIIIIIIIIIII
- 254.0 * IIIIIZIIIIIIIIIIIIII
- 254.0 * IIIIZIIIIIIIIIIIIIII
- 254.0 * IIIZIIIIIIIIIIIIIIII
- 254.0 * IIZIIIIIIIIIIIIIIIII
- 254.0 * IZIIIIIIIIIIIIIIIIII
- 254.0 * ZIIIIIIIIIIIIIIIIIII
+ 265.0 * IIIIIIIIIIIIIIIIZIIZ
+ 508.0 * IIIIIIIIIIIIIIIIZIZI
+ 254.0 * IIIIIIIIIIIIIIIIZZII
+ 11.0 * IIIIIIIIIIIIIIIZIIZI
+ 265.0 * IIIIIIIIIIIIIIIZIZII
+ 508.0 * IIIIIIIIIIIIIIZIIIZI
+ 508.0 * IIIIIIIIIIIIIIZIIZII
+ 762.0 * IIIIIIIIIIIIIIZIZIII
+ 254.0 * IIIIIIIIIIIIIZIIIIIZ
+ 254.0 * IIIIIIIIIIIIIZIIIZII
+ 254.0 * IIIIIIIIIIIIIZIIZII

In [48]:
from qiskit.utils import algorithm_globals
from qiskit.algorithms.minimum_eigensolvers import QAOA, SamplingVQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)

In [53]:
from qiskit_ibm_runtime import Sampler, Session, Options
from qiskit_ibm_runtime import QiskitRuntimeService
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="my_token",  overwrite=True)

In [54]:
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_qasm_simulator")

In [55]:
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=Sampler(session=backend), optimizer=COBYLA(), initial_point=[0.0, 0.0])

In [56]:
qaoa = MinimumEigenOptimizer(qaoa_mes)

In [59]:
qaoa_result = qaoa.solve(qaoa_model)

In [60]:
print(qaoa_result.prettyprint())

objective function value: 13.0
variable values: q0=1.0, q1=1.0, q2=1.0, q3=0.0, q4=0.0, q5=0.0, q6=0.0, q7=0.0
status: SUCCESS
